# Analysis


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fiona
from shapely.geometry import MultiPoint, Point, Polygon,shape
import random
import warnings

## Data Cleaning and preprocessing

In [28]:
calc_df_fixed=pd.read_csv('../outputs/calc_fixed_df.csv', low_memory=False)
mm_df=pd.read_csv('../outputs/mm_dataset.csv')
#removing all non geolocated services
calc_df_fixed = calc_df_fixed.drop(calc_df_fixed[calc_df_fixed['long'].isna()].index)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

Cleaning all housing units outside of comune di milano

In [ ]:
multipol = fiona.open(r"../datasets/milan_shape/L090102_ComuneMilano.shp")
multi = next(iter(multipol))

found=False
while not found:
    a=random.uniform(8.8, 9.37)
    b=random.uniform(45.35, 45.9)
    point = Point(a, b)
    found=point.within(shape(multi['geometry']))

print(a, b)

multipol = fiona.open(r"C:\Users\Jordi\Downloads\ESP_adm_shp\ESP_adm0.shp")
multi = next(iter(multipol))

point = Point(0,42)
point.within(shape(multi['geometry']))

## Summary statistics

In [ ]:
desc_mm=calc_df_fixed.groupby('df_name').describe()
desc_mm=desc_mm.drop(['long', 'lat'], axis=1)
desc_mm=desc_mm.T
desc_mm.head()

In [ ]:
min_services=calc_df_fixed.groupby('df_name').min()
min_services=min_services.drop(['long', 'lat', 'cat1_name', 'cat2_name'], axis=1)
desc_services=pd.DataFrame()
desc_services['min']=min_services.min(axis=1)
desc_services['id_min']=min_services.idxmin(axis=1)
max_services=calc_df_fixed.groupby('df_name').min()
max_services=max_services.drop(['long', 'lat', 'cat1_name', 'cat2_name'], axis=1)
desc_services['max']=max_services.max(axis=1)
desc_services['id_max']=max_services.idxmax(axis=1)
avg_services=calc_df_fixed.groupby('df_name').mean()
desc_services['avg']=avg_services.mean(axis=1)
sd_services=calc_df_fixed.groupby('df_name').std()
desc_services['sd']=avg_services.std(axis=1)
desc_services

In [ ]:
max_20=calc_df_fixed.groupby('df_name').min()
max_20=max_20.drop(['long', 'lat', 'cat1_name', 'cat2_name'], axis=1)
max_20=max_20.T
max_20

In [ ]:
plt.hist(max_20['ss2'])

# Index calculator

We construct the index as illustrated in the paper, note that in this notebook we just prove the concept, the full calculation happnes in another file (index.py)

In [71]:
def penality_function(x, dhat):
    if x <= dhat:
        return 1
    if x > dhat:
        return 1/(x-dhat+1)

def con_index(dist_list, dhat, n_max, alpha, beta):
    if  alpha + beta != 1:
        raise Exception("Alpha and Beta should sum to one")
    d_min = min(dist_list)
    close_list=[d for d in dist_list if d <=dhat]
    n = len(close_list)
    CI = alpha*penality_function(d_min, dhat)+beta*(n/n_max)
    return CI

con_index([9,6,6,6,6], 5, 3, 0.5, 0.5)

0.25

In [57]:
def best_in_class(df_name, dhat, df=calc_df_fixed):
    e_list=list(df.columns)
    e_list=[item for item in e_list if item not in ['name', 'long', 'lat','df_name', 'cat1', 'cat2', 'cat1_name', 'cat2_name']]
    nmax_dict=dict.fromkeys(e_list)
    if df_name=='all':
        pass
    else:
        df=df[df.df_name==df_name]
    for elem in e_list:
        dist_list=list(df[str(elem)])
        close_list=[d for d in dist_list if d <=dhat]
        nmax_dict[elem]=len(close_list)
    return max(nmax_dict.values())

best_in_class('biblio', 60*15)

3

In [ ]:
min(calc_df_fixed['10002602'].to_list())

In [97]:
mm_columns_list=['CODICE EDIFICIO', 'full_address', 'lat', 'lon']
index_df=mm_df[mm_columns_list]
index_df

,CODICE EDIFICIO,full_address,lat,lon
0,10001708,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
1,10001709,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
2,10001710,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
3,10001711,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053
4,10001712,VIA DEL TURCHINO 22 MILANO,45.451893,9.228053
...,...,...,...,...
975,51039405,VIA SENIGALLIA 60/G MILANO,45.532143,9.168970
976,51039406,VIA SENIGALLIA 60 MILANO,45.532143,9.168970
977,51039407,VIA SENIGALLIA 60 MILANO,45.532143,9.168970
978,51039408,VIA SENIGALLIA 60 MILANO,45.532143,9.168970


In [78]:
columns_list=list(calc_df_fixed.df_name.unique())+['all']
[best_in_class(name, 60*15)  for name in columns_list] #checking if function works for all dataframes

['acqua',
 'biblio',
 'ciclabili',
 'consu',
 'cult',
 'edicole',
 'farmacie',
 'metro',
 'parchi',
 'posta',
 'serd',
 'sinf',
 'sita',
 'sport',
 'sprim',
 'ss2',
 'ssec',
 'treni',
 'uni',
 'wifi',
 'distr',
 'all']

In [86]:
def fun_run(codice_edificio, dhat, alpha, beta, out_df, df=calc_df_fixed):
    columns_list=list(df.df_name.unique())+['all']
    #out_dict
    out_list=[]
    col_out_list=[]
    for column in columns_list:
        if column=='all':
            df_subset=df

        else:
            df_subset=df[df.df_name==column]
        dist_list=df_subset[str(codice_edificio)].to_list()
        if dist_list==[]:
            out=np.nan
            warnings.warn("Empty list")
        else:
            n_max=best_in_class(column, dhat)
            try:
                out=con_index(dist_list=dist_list, dhat=dhat, n_max=n_max, alpha=alpha, beta=beta)
            except:
                out=np.nan
                warnings.warn("Index not calculated because of some issue")
        col_out_list.append(column)
        out_list.append(out)
    if col_out_list == ['acqua', 'biblio', 'ciclabili','consu','cult', 'edicole', 'farmacie', 'metro', 'parchi', 'posta', 'serd', 'sinf', 'sita', 'sport', 'sprim', 'ss2', 'ssec', 'treni', 'uni', 'wifi', 'distr','all']:
        return out_list
    else:
        raise Exception("Not all columns are outputs of fun run")


fun_run(10003003, dhat=60*15, alpha=0.5, beta=0.5, out_df=index_df)

[1.0,
 0.8333333333333333,
 0.5570175438596491,
 0.8,
 0.5106382978723404,
 0.6120689655172413,
 0.7580645161290323,
 0.6,
 0.5982658959537572,
 0.6875,
 0.6,
 0.8928571428571428,
 0.5357142857142857,
 0.7586206896551724,
 0.85625,
 0.5833333333333334,
 0.686046511627907,
 0.0027716186252771603,
 0.5263157894736842,
 0.5943396226415094,
 0.547085201793722,
 0.6635044642857143]

In [98]:
index_df=index_df.head(n=5)
index_df['list']=index_df['CODICE EDIFICIO'].apply(lambda x: fun_run(x, dhat=60*15, alpha=0.5, beta=0.5, out_df=index_df))
index_df

C:\Users\iodio\AppData\Local\Temp\ipykernel_45416\1734720053.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_df['list']=index_df['CODICE EDIFICIO'].apply(lambda x: fun_run(x, dhat=60*15, alpha=0.5, beta=0.5, out_df=index_df))


,CODICE EDIFICIO,full_address,lat,lon,list
0,10001708,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627..."
1,10001709,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627..."
2,10001710,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627..."
3,10001711,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627..."
4,10001712,VIA DEL TURCHINO 22 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627..."


In [99]:
list_df=pd.DataFrame(index_df['list'].to_list(), columns = columns_list)
index_df = pd.concat([index_df, list_df], axis=1)
index_df

,CODICE EDIFICIO,full_address,lat,lon,list,acqua,biblio,ciclabili,consu,cult,...,sita,sport,sprim,ss2,ssec,treni,uni,wifi,distr,all
0,10001708,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
1,10001709,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
2,10001710,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
3,10001711,VIA DEL TURCHINO 20 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
4,10001712,VIA DEL TURCHINO 22 MILANO,45.451893,9.228053,"[0.6666666666666666, 0.6666666666666666, 0.627...",0.666667,0.666667,0.627193,0.001999,0.000592,...,0.571429,0.594828,0.63125,0.541667,0.593023,0.625,0.064103,0.566038,0.515695,0.614397
